In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

api_key = os.environ["LANGCHAIN_API"] 

gemini_api = os.environ["GEMINI_API"]

In [2]:
from langchain_google_genai import GoogleGenerativeAI

In [3]:
llm = GoogleGenerativeAI(
    model="gemini-2.5-flash-lite"
)

## Part - 5 Multi Query

In [4]:
#load blog
 
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs= dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    ),
)

blog_docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# split

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)

splits= text_splitter.split_documents(blog_docs)


In [6]:
#index 

from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_community.vectorstores import Chroma


embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001", 
    api_key=gemini_api,
    credentials= None
)

vectorstore = Chroma.from_documents(documents = splits,
                                   embedding = embeddings)

retriever = vectorstore.as_retriever()

In [7]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are an AI language model assistant, Your takse is to generate five different versions of the given user questions and to retrieve relevant documents from the database. By generating multiple different perspectives on the user question, your goal is to make the user overcome some of the limitations of the distance based similarity search, provide these alternatives responses separated by newlines. Original question {question}"""

prompt_pers = ChatPromptTemplate.from_template(template)

In [8]:
from langchain_core.output_parsers import StrOutputParser

generate_queries = (
    prompt_pers 
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [9]:
from langchain_core.load import dumps, loads

def get_uniq(documents : list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))

    return [loads(doc) for doc in unique_docs]

In [10]:
#retrieve

question = "what is task decomposition for LLM agents ?"
retrieval_chain = generate_queries | retriever.map() | get_uniq
docs = retrieval_chain.invoke({"question": question})
len(docs)

C:\Users\dhruv\AppData\Local\Temp\ipykernel_5152\4290818311.py:7: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


4

In [11]:
from operator import itemgetter

#RAG 

template = """answer the following question based on this context:
        {context}
        
        Question : {question}"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()

)

In [12]:
final_rag_chain.invoke({"question" : question})

'Task decomposition for LLM agents refers to the process of breaking down large, complex tasks into smaller, more manageable subgoals. This allows the agent to handle intricate tasks more efficiently. This decomposition can be achieved through several methods:\n\n*   **Simple prompting:** Using instructions like "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?".\n*   **Task-specific instructions:** Providing directives tailored to the task, such as "Write a story outline." for novel writing.\n*   **Human inputs:** Direct guidance from a human user.\n*   **Chain of Thought (CoT):** Instructing the model to "think step by step" to decompose difficult tasks into simpler ones.\n*   **Tree of Thoughts (ToT):** This approach extends CoT by exploring multiple reasoning possibilities at each step, generating several thoughts per step to create a tree-like structure of problem-solving.\n*   **External classical planners:** Using tools like the Planning Domain Definition Lang

## Part 6 : RAG Fusion

In [13]:
template = """You are an AI language model assistant, Your task is to generate five different versions of the given user questions and to retrieve relevant documents from the database. By generating multiple different perspectives on the user question, your goal is to make the user overcome some of the limitations of the distance based similarity search, provide these alternatives responses separated by newlines. Original question {question}"""

prompt_pers = ChatPromptTemplate.from_template(template)

In [14]:
def reciprocal_rank_fusion(results: list[list], k = 60):
    """reciprocal rank fusion function that takes in multiple lists from ranked documents and an optional paramater k is used in the RRF formula"""

    fused_scores = {}

    for rank, doc in enumerate(docs):
        doc_str = dumps(doc)

        if doc_str not in fused_scores:
            fused_scores[doc_str] = 0
        
        prev_score = fused_scores[doc_str]
        fused_scores[doc_str] += 1 / (rank + k)
    
    reranked_res = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key = lambda x: x[1], reverse = True)
    ]

    return reranked_res

ret_chain_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = ret_chain_fusion.invoke({"question": question})
len(docs)

4

In [17]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": ret_chain_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

final_rag_chain.invoke({"question":question})

['Task decomposition for LLM agents involves breaking down large, complex tasks into smaller, more manageable subgoals. This allows the agent to handle intricate tasks more efficiently. Methods for task decomposition include:',
 '',
 '*   **LLM with simple prompting:** Using prompts like "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?".',
 '*   **Task-specific instructions:** Providing instructions tailored to the specific task, such as "Write a story outline." for novel writing.',
 '*   **Human inputs:** Relying on human guidance to define subgoals.',
 '*   **Chain of Thought (CoT):** Instructing the model to "think step by step" to decompose hard tasks into simpler ones, which also provides insight into the model\'s thinking process.',
 '*   **Tree of Thoughts (ToT):** Extending CoT by exploring multiple reasoning possibilities at each step, generating several thoughts per step to create a tree structure, which can be searched using BFS or DFS.']

## Part - 7 Decomposition

In [18]:
from langchain_core.prompts import ChatPromptTemplate

#Decomposition 

template = """you are a helpful assistant that generates multiples sub questions related to an input question. \n
the goal is to break the input into a set of sub questions to help the user understand his requirements better and for the agent to be able to answer all of them \n
generate multiple queries related to the question " {question} \n
please output (3 queries)"""

prompt_dc = ChatPromptTemplate.from_template(template)

In [19]:
generate_queries_dec = (prompt_dc | llm | StrOutputParser() | (lambda x: x.split("\n")))

question = "what are the components of a LLM system ?"
questions = generate_queries_dec.invoke({"question": question})

In [20]:
questions

['Here are 3 sub-questions related to "What are the components of an LLM system?":',
 '',
 '1.  What are the core architectural building blocks that constitute a Large Language Model (LLM)?',
 '2.  Beyond the model itself, what other software and hardware elements are typically involved in deploying and running an LLM system?',
 '3.  What are the key data-related components necessary for training and operating an LLM?']

 Answer recursively

In [21]:
template_dec = """here is the question you need to answer:
                \n \n {question} \n \n
                here is any available bg question + answer pairs:
                \n \n {q_a_pairs} \n \n
                here is additional context relevant to this query:
                \n \n {context} \n \n
                use the above context and question answer pairs to ans this query in a better manner"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [22]:
def format_qa_pair(question, answer):
    """format Q and A pair"""

    formatted_str = ""
    formatted_str += f"Question: {question} \n Answer: {answer} \n \n"
    return formatted_str.strip()

In [23]:

questions = [
    q.strip()  
    for q in questions
    if q and q.strip()  
]


In [27]:
q_a_pairs = ""

for q in questions:
    rag_chain = (
        {"context": itemgetter("question") | retriever,
         "question": itemgetter("question"),
         "q_a_pairs": itemgetter("q_a_pairs")}
         | decomposition_prompt
         | llm
         | StrOutputParser()
    )

    answer = final_rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [28]:
answer

['Based on the provided context, the question about "key data-related components necessary for training and operating an LLM" cannot be answered. The text discusses LLM-powered autonomous agents, planning, task decomposition, and self-reflection, but it does not delve into the specifics of data requirements for training or operating LLMs.']

Answer individually

In [32]:
from langchain_core.runnables import RunnableLambda
from langsmith import Client

In [34]:
hub = Client()
prompt_rag = hub.pull_prompt("rlm/rag-prompt")

In [40]:
import langchain
print(langchain.__version__)


1.0.0


In [45]:
def retrieve_and_rag(question, prompt, sub_ques_gen_chain):
    """rag on each sub question"""

    sub_ques = sub_ques_gen_chain.invoke({"question": question})
    rag_res = []

    for sub_qu in sub_ques:
        if not sub_qu or sub_qu.strip():
            continue
        retrieved_docs = retriever.invoke(sub_qu)
        answer = (prompt | llm | StrOutputParser()).invoke({"context": retrieved_docs, "question": sub_qu})

        rag_res.append(answer)
    
    return rag_res, sub_ques

answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_dec)

In [48]:
def format_qa_pairs(questions, answers):
    """format question and answer pairs"""

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start = 1):
        formatted_string += f"Question {i}: {question} \n answer {i} \n \n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

template = """Here is a set of QA pairs:
{context}

use these to synthesize a answers to the question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context" :context , "question": question})

"Here's a synthesis of the components of an LLM system based on the provided QA pairs:\n\nAn LLM system is composed of several key components that work together to process and generate human-like text. The core of the system is the **Large Language Model (LLM)** itself, which is a complex neural network trained on a massive dataset of text and code. This training allows the LLM to learn patterns, grammar, facts, reasoning abilities, and more.\n\nTo interact with the LLM and provide it with context for a specific task, an **input prompt** is used. This prompt guides the LLM's output by specifying the desired action, providing relevant information, or setting constraints.\n\nThe LLM then processes this input and generates a **response**. This response is the output of the system, designed to be coherent, relevant, and informative based on the prompt and the LLM's training.\n\nIn some advanced LLM systems, there might be additional components for **fine-tuning**. This process involves fur

###  Part - 8 step back 

In [50]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        "input" : "could the members of the police perform lawful arrests ?",
        "output" : "what can be the members of the police do ?",
    },
    {
        "input": "Jan sindel was born in what country ?",
        "output": "what is jan sindel personal history ?",
    },
]

#We now transform these to example messages

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples = examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),  
    ]
)

In [52]:
generate_queries_step_back = prompt | llm | StrOutputParser()
question = "What is task decomposiion for a LLM agent?"
generate_queries_step_back.invoke({"question": question})

'AI: What is task decomposition?'

In [53]:
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""

response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        "step_back_context": generate_queries_step_back | retriever,
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for a LLM agent refers to the process of breaking down a large, complex task into smaller, more manageable subgoals. This is a crucial component of planning in LLM-powered autonomous agents, enabling them to efficiently handle intricate problems.\n\nHere\'s a breakdown of how task decomposition is approached:\n\n*   **Purpose:** The primary goal is to transform a big, overwhelming task into a series of simpler, achievable steps. This not only makes the task easier to manage but also provides insights into the agent\'s reasoning process.\n\n*   **Methods of Decomposition:**\n    *   **Chain of Thought (CoT):** This is a standard prompting technique where the LLM is instructed to "think step by step." By utilizing more computation during inference, the model can decompose complex tasks into a sequence of simpler ones. This method helps in interpreting the model\'s thought process.\n    *   **Tree of Thoughts (ToT):** An extension of CoT, ToT explores multiple reasonin

### Part - 9 HyDE

In [54]:
from langchain_core.prompts import ChatPromptTemplate

template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""

prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser()
)

question = "what is task decompositions for llm agents"
generate_docs_for_retrieval.invoke({"question": question})

'Here\'s a passage answering "What are task decompositions for LLM agents?" in a scientific paper style:\n\n---\n\n**Passage:**\n\nTask decomposition for Large Language Model (LLM) agents refers to the process of breaking down a complex, overarching objective into a sequence of smaller, more manageable, and executable sub-tasks. LLMs, while possessing impressive generative capabilities and a broad understanding of language and concepts, often struggle with intricate, multi-step problems that require planning, reasoning, and interaction with external tools or environments. Task decomposition addresses this limitation by enabling the LLM to approach a large task in a structured and iterative manner. This involves identifying distinct stages or components of the desired outcome and formulating them as individual prompts or instructions that the LLM can process and execute. The output of one sub-task often serves as the input or context for the next, creating a dependency chain. This hiera

In [55]:
retrieval_chain = generate_docs_for_retrieval | retriever

retrieved_docs = retrieval_chain.invoke({"question" : question})
retrieved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.\nAnother quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) translates the PDDL plan back into natural language. Essentially, the planning step is outsourced to an external tool, assuming the availability of domain-specific PDDL and a suitable planner

In [56]:


# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retrieved_docs,"question":question})



'Task decomposition for LLM agents refers to the process of breaking down a complex task into smaller, more manageable subgoals. This enables the agent to handle intricate tasks more efficiently.\n\nThis decomposition can be achieved in several ways:\n\n*   **LLM with simple prompting:** This involves using straightforward prompts like "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?".\n*   **Task-specific instructions:** Providing instructions tailored to the specific task, such as "Write a story outline." for novel writing.\n*   **Human inputs:** Receiving guidance from humans to define the subtasks.\n*   **Chain of Thought (CoT):** Instructing the LLM to "think step by step" to decompose hard tasks into simpler steps, which also provides insight into the model\'s reasoning.\n*   **Tree of Thoughts (ToT):** Extending CoT by exploring multiple reasoning possibilities at each step. It decomposes the problem into thought steps and generates multiple thoughts per step